<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_1_SRAL_data_access_HDA_Data_Store.ipynb"><< Accessing SRAL data</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_3_SRAL_coverage.ipynb">Determining SRAL product coverage >></a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT

<div class="alert alert-block alert-success">
<h3>Learn SRAL: Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
The following modules are prerequisites for this notebook:
- **<a href="1_1_SRAL_data_access_HDA_Data_Store.ipynb" target="_blank">1_1_SRAL_data_access_HDA_Data_Store.ipynb</a>**; this will download the required SRAL products for this notebook.

</div>
<hr>

# 1.2 Understanding SRAL product structure

### Data used

| Product Description  | Data Store collection ID| Product Navigator | WEkEO HDA ID | WEkEO metadata |
|:--------------------:|:-----------------------:|:-------------:|:-----------------:|:--------:|
| Sentinel-3 SRAL level-1B | EO:EUM:DAT:0406 |<a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:SR_1_SRA___NTC?query=SRAL&s=advanced" target="_blank">link</a> | EO:EUM:DAT:SENTINEL-3:SR_1_SRA___ | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3ASR_1_SRA___" target="_blank">link</a> |
| Sentinel-3 SRAL level-2 | EO:EUM:DAT:0415 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:SR_2_WAT___NTC?query=SRAL&s=advanced" target="_blank">link</a> | EO:EUM:DAT:SENTINEL-3:SR_2_WAT___ | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AEUM%3ADAT%3ASENTINEL-3%3ASR_2_WAT___" target="_blank">link</a>|

### Learning outcomes

At the end of this notebook you will know;
* What the SAFE format is
* What components are inside a SAFE format file from SRAL
* What variables are in present in each component for SRAL data files (level-1B and level-2)

### Outline

Data from all the Sentinel satellites operated under the European Commissions Copernicus Programme are delievered in "**SAFE format**". The Sentinel-SAFE format is a specific variation of the Standard Archive Format for Europe (SAFE) format specification designed for the Sentinel satellite products.  It is based on the XML Formatted Data Units (XFDU) standard under development by the Consultative Committee for Space Data Systems (CCSDS). Sentinel-SAFE is a profile of XFDU, and it restricts the XFDU specifications for specific utilisation in the Earth Observation domain, providing semantics in the same domain to improve interoperability between ground segment facilities.


Each product package includes:

* a manifest file containing a metadata section and a data object section (an xml file).

* measurement data files (NetCDF-4 format)

* annotation data files, if defined (NetCDF-4 format)

The product package can exist as a directory in a filesystem, zipped folder or tarball.

The naming of the Sentinel-SAFE files follows a specific convention that you can learn about <a href="https://eumetsatspace.atlassian.net/wiki/spaces/DPF/pages/edit-v2/1597702263?" target="_blank">here</a>

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
 1. [Querying SRAL file structure (Level-1B)](#section1)
 2. [Querying SRAL file structure (Level-2)](#section2)
 3. [Applying your knowledge](#section3)

<hr>

In [1]:
# library imports
import glob
import inspect
from IPython.display import display, Markdown
import os
import xarray as xr
import xml.etree.ElementTree as ET
import warnings

warnings.filterwarnings('ignore')

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Querying SRAL file structure (Level-1B)
[Back to top](#TOC_TOP)

</div>

First we will create a variable which holds the path for the file we are interested in, in this case the level-1B SRAL file that we downloaded in the <a href="./1_1_SRAL_data_access_HDA_Data_Store.ipynb" target="_blank"> SRAL data access HDA Data Store</a> notebook.

In [2]:
# selecting SAFE directory
SAFE_directory = os.path.join(os.getcwd(), 'products', 
    'S3B_SR_1_SRA____20220218T110359_20220218T115429_20220220T023624_3029_062_365______MAR_O_ST_004.SEN3')

Next, we'll create another variable that takes this path, and finds and adds on the names of the manifest file within the SAFE folder.  

In [3]:
# selecting SAFE manifest
SAFE_manifest = glob.glob(os.path.join(SAFE_directory, 'xfd*.xml'))[0]
display(Markdown('**Manifest file:** {}'.format(os.path.basename(SAFE_manifest))))

**Manifest file:** xfdumanifest.xml

Then, we'll read the manifest file in to a Python object format called a dictionary. This will make it easier for us to interact with the product in the next step.

In [4]:
# Reading SAFE manifest and outputting/collecting component names
tree = ET.parse(SAFE_manifest)
root = tree.getroot()
items = root.find('dataObjectSection')
SAFE_components = []
for item in items:
    display(Markdown("**File name:** {}".format(item[0][0].get('href'))))
    display(Markdown("*File info: {}*".format(item[0][0].get('textInfo'))))
    SAFE_components.append(os.path.join(SAFE_directory, item[0][0].get('href')))

**File name:** measurement.nc

*File info: Measurement Data Object File*

**File name:** acquisition.nc

*File info: Acquisition Data Object File*

The next section of code will display each of the components within the product. Scrolling through you can see the main groups of variables, including some you may already know or be interested in working with.

In [5]:
# Display component structure and variables
for SAFE_component in sorted(SAFE_components):
    ds = xr.open_dataset(SAFE_component, decode_times=False)
    display(Markdown('**Filename:** {}'.format(os.path.basename(SAFE_component))))
    display(ds)
    ds.close()

**Filename:** acquisition.nc

<xarray.Dataset>
Dimensions:                                (time_l1b_acq: 16)
Coordinates:
  * time_l1b_acq                           (time_l1b_acq) float64 6.985e+08 ....
    lat_l1b_acq                            (time_l1b_acq) float64 ...
    lon_l1b_acq                            (time_l1b_acq) float64 ...
Data variables:
    isp_coarse_time_l1b_acq                (time_l1b_acq) float64 ...
    isp_fine_time_l1b_acq                  (time_l1b_acq) float64 ...
    sral_fine_time_l1b_acq                 (time_l1b_acq) float64 ...
    alt_l1b_acq                            (time_l1b_acq) float64 ...
    orb_alt_rate_l1b_acq                   (time_l1b_acq) float32 ...
    surf_type_l1b_acq                      (time_l1b_acq) float32 ...
    flag_positioning_phase_acq             (time_l1b_acq) float32 ...
    flag_search_phase_acq                  (time_l1b_acq) float32 ...
    positioning_phase_acq_valid_cyles_ind  (time_l1b_acq) float32 ...
    flag_locking_phase_acq                 (time_l1b_acq) float32 ...
    locking_phase_acq_valid_cyles_ind      (time_l1b_acq) float32 ...
Attributes: (12/29)
    Conventions:                      CF-1.6
    title:                            IPF SRAL Level 1 Measurement
    processing_baseline:              Unavailable_PB_ID
    mission_name:                     Sentinel 3B
    altimeter_sensor_name:            SRAL
    gnss_sensor_name:                 GNSS
    ...                               ...
    source:                           IPF-SR-1 06.21
    history:                           
    contact:                          ops@eumetsat.int
    creation_time:                    2022-02-20T02:41:07Z
    first_meas_time:                  2022-02-18T11:03:59.187431Z
    last_meas_time:                   2022-02-18T11:54:28.760871Z

**Filename:** measurement.nc

<xarray.Dataset>
Dimensions:                              (echo_sample_ind: 128,
                                          max_multi_stack_ind: 256,
                                          time_l1b_echo_lrm: 0,
                                          time_l1b_echo_sar_ku: 60772,
                                          time_l1b_echo_plrm: 59351)
Coordinates:
  * echo_sample_ind                      (echo_sample_ind) int8 0 1 ... 126 127
  * max_multi_stack_ind                  (max_multi_stack_ind) int16 0 1 ... 255
  * time_l1b_echo_lrm                    (time_l1b_echo_lrm) float64 
    lat_l1b_echo_lrm                     (time_l1b_echo_lrm) float64 ...
    lon_l1b_echo_lrm                     (time_l1b_echo_lrm) float64 ...
  * time_l1b_echo_sar_ku                 (time_l1b_echo_sar_ku) float64 6.985...
    lat_l1b_echo_sar_ku                  (time_l1b_echo_sar_ku) float64 ...
    lon_l1b_echo_sar_ku                  (time_l1b_echo_sar_ku) float64 ...
  * time_l1b_echo_plrm                   (time_l1b_echo_plrm) float64 6.985e+...
    lat_l1b_echo_plrm                    (time_l1b_echo_plrm) float64 ...
    lon_l1b_echo_plrm                    (time_l1b_echo_plrm) float64 ...
Data variables: (12/175)
    UTC_day_l1b_echo_lrm                 (time_l1b_echo_lrm) float32 ...
    UTC_sec_l1b_echo_lrm                 (time_l1b_echo_lrm) float64 ...
    GPS_time_l1b_echo_lrm                (time_l1b_echo_lrm) float64 ...
    isp_coarse_time_l1b_echo_lrm         (time_l1b_echo_lrm) float64 ...
    isp_fine_time_l1b_echo_lrm           (time_l1b_echo_lrm) float64 ...
    sral_fine_time_l1b_echo_lrm          (time_l1b_echo_lrm) float64 ...
    ...                                   ...
    agc_cor_c_l1b_echo_plrm              (time_l1b_echo_plrm) float64 ...
    sig0_cal_ku_l1b_echo_plrm            (time_l1b_echo_plrm) float64 ...
    sig0_cal_c_l1b_echo_plrm             (time_l1b_echo_plrm) float64 ...
    i2q2_meas_ku_l1b_echo_plrm           (time_l1b_echo_plrm, echo_sample_ind) float64 ...
    i2q2_meas_c_l1b_echo_plrm            (time_l1b_echo_plrm, echo_sample_ind) float64 ...
    stack_mask_vector_l1b_echo_sar_ku    (time_l1b_echo_sar_ku, max_multi_stack_ind) float32 ...
Attributes: (12/33)
    Conventions:                      CF-1.6
    title:                            IPF SRAL Level 1 Measurement
    processing_baseline:              Unavailable_PB_ID
    mission_name:                     Sentinel 3B
    altimeter_sensor_name:            SRAL
    gnss_sensor_name:                 GNSS
    ...                               ...
    source:                           IPF-SR-1 06.21
    history:                           
    contact:                          ops@eumetsat.int
    creation_time:                    2022-02-20T02:41:07Z
    first_meas_time:                  2022-02-18T11:03:59.187431Z
    last_meas_time:                   2022-02-18T11:54:28.760871Z

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Querying SRAL file structure (Level-2)
[Back to top](#TOC_TOP)

</div>

Now let's look at the level-2 SRAL product that we downloaded previously.

In [6]:
# selecting SAFE directory
SAFE_directory = os.path.join(os.getcwd(), 'products', 
    'S3B_SR_2_WAT____20220218T110400_20220218T115102_20220220T035329_2822_062_365______MAR_O_ST_004.SEN3')

Again, we'll create another variable that takes this path, and finds and adds on the names of the manifest file within the SAFE folder.  

In [7]:
# selecting SAFE manifest
SAFE_manifest = glob.glob(os.path.join(SAFE_directory, 'xfd*.xml'))[0]
display(Markdown('**Manifest file:** {}'.format(os.path.basename(SAFE_manifest))))

**Manifest file:** xfdumanifest.xml

Then, we'll read the manifest file for this level-2 data product in to another dictionary. 

In [8]:
# Reading SAFE manifest and outputting/collecting component names
tree = ET.parse(SAFE_manifest)
root = tree.getroot()
items = root.find('dataObjectSection')
SAFE_components = []
for item in items:
    display(Markdown("**File name:** {}".format(item[0][0].get('href'))))
    display(Markdown("*File info: {}*".format(item[0][0].get('textInfo'))))
    SAFE_components.append(os.path.join(SAFE_directory, item[0][0].get('href')))

**File name:** enhanced_measurement.nc

*File info: Measurement Data Object File*

**File name:** standard_measurement.nc

*File info: Measurement Data Object File*

**File name:** reduced_measurement.nc

*File info: Measurement Data Object File*

Finally, let's display the components of the product...

In [9]:
# Display component structure and variables
for SAFE_component in sorted(SAFE_components):
    ds = xr.open_dataset(SAFE_component)
    display(Markdown('**Filename:** {}'.format(os.path.basename(SAFE_component))))
    display(ds)
    ds.close()

**Filename:** enhanced_measurement.nc

<xarray.Dataset>
Dimensions:                                        (time_01: 2799,
                                                    time_20_ku: 56300,
                                                    time_20_c: 54882,
                                                    echo_sample_ind: 128)
Coordinates:
  * time_01                                        (time_01) datetime64[ns] 2...
  * time_20_ku                                     (time_20_ku) datetime64[ns] ...
  * time_20_c                                      (time_20_c) datetime64[ns] ...
  * echo_sample_ind                                (echo_sample_ind) int8 0 ....
    lat_01                                         (time_01) float64 ...
    lon_01                                         (time_01) float64 ...
    lat_20_ku                                      (time_20_ku) float64 ...
    lon_20_ku                                      (time_20_ku) float64 ...
    lat_20_c                                       (time_20_c) float64 ...
    lon_20_c                                       (time_20_c) float64 ...
Data variables: (12/367)
    UTC_day_01                                     (time_01) datetime64[ns] ...
    UTC_sec_01                                     (time_01) float64 ...
    UTC_day_20_ku                                  (time_20_ku) datetime64[ns] ...
    UTC_sec_20_ku                                  (time_20_ku) float64 ...
    UTC_day_20_c                                   (time_20_c) datetime64[ns] ...
    UTC_sec_20_c                                   (time_20_c) float64 ...
    ...                                             ...
    num_20hz_meas_01_ku                            (time_01) float32 ...
    num_20hz_meas_01_c                             (time_01) float32 ...
    orbit_type_01                                  (time_01) float32 ...
    waveform_qual_ice_20_ku                        (time_20_ku) float32 ...
    iono_cor_alt_filtered_01_ku                    (time_01) float32 ...
    iono_cor_alt_filtered_01_plrm_ku               (time_01) float32 ...
Attributes: (12/45)
    Conventions:                       CF-1.6
    title:                             IPF SRAL/MWR Level 2 Measurement
    mission_name:                      Sentinel 3B
    altimeter_sensor_name:             SRAL
    radiometer_sensor_name:            MWR
    gnss_sensor_name:                  GNSS
    ...                                ...
    xref_iono_data:                    S3B_SR_2_RGI_AX_20220218T000000_202202...
    xref_mog2d_data:                   S3__SR_2_PMO_AX_20220218T060000_202202...
    xref_seaice_concentration:         S3__SR_2_SIC_AX_20220209T000000_202202...
    xref_altimeter_ltm:                S3B_SR_1_CA1LAX_20000101T000000_202202...
    xref_doris_uso:                    S3B_SR_1_USO_AX_20180501T144459_202202...
    xref_time_correlation:             S3B_AX___FRO_AX_20220216T000000_202202...

**Filename:** reduced_measurement.nc

<xarray.Dataset>
Dimensions:                             (time_01: 2799)
Coordinates:
  * time_01                             (time_01) datetime64[ns] 2022-02-18T1...
    lat_01                              (time_01) float64 ...
    lon_01                              (time_01) float64 ...
Data variables: (12/48)
    UTC_day_01                          (time_01) datetime64[ns] ...
    UTC_sec_01                          (time_01) float64 ...
    surf_type_01                        (time_01) float32 ...
    surf_class_01                       (time_01) float32 ...
    rad_surf_type_01                    (time_01) float32 ...
    dist_coast_01                       (time_01) float64 ...
    ...                                  ...
    rain_flag_01_plrm_ku                (time_01) float32 ...
    open_sea_ice_flag_01_ku             (time_01) float32 ...
    open_sea_ice_flag_01_plrm_ku        (time_01) float32 ...
    orbit_type_01                       (time_01) float32 ...
    iono_cor_alt_filtered_01_ku         (time_01) float32 ...
    iono_cor_alt_filtered_01_plrm_ku    (time_01) float32 ...
Attributes: (12/45)
    Conventions:                       CF-1.6
    title:                             IPF SRAL/MWR Level 2 Measurement
    mission_name:                      Sentinel 3B
    altimeter_sensor_name:             SRAL
    radiometer_sensor_name:            MWR
    gnss_sensor_name:                  GNSS
    ...                                ...
    xref_iono_data:                    S3B_SR_2_RGI_AX_20220218T000000_202202...
    xref_mog2d_data:                   S3__SR_2_PMO_AX_20220218T060000_202202...
    xref_seaice_concentration:         S3__SR_2_SIC_AX_20220209T000000_202202...
    xref_altimeter_ltm:                S3B_SR_1_CA1LAX_20000101T000000_202202...
    xref_doris_uso:                    S3B_SR_1_USO_AX_20180501T144459_202202...
    xref_time_correlation:             S3B_AX___FRO_AX_20220216T000000_202202...

**Filename:** standard_measurement.nc

<xarray.Dataset>
Dimensions:                                        (time_01: 2799,
                                                    time_20_ku: 56300,
                                                    time_20_c: 54882)
Coordinates:
  * time_01                                        (time_01) datetime64[ns] 2...
  * time_20_ku                                     (time_20_ku) datetime64[ns] ...
  * time_20_c                                      (time_20_c) datetime64[ns] ...
    lat_01                                         (time_01) float64 ...
    lon_01                                         (time_01) float64 ...
    lat_20_ku                                      (time_20_ku) float64 ...
    lon_20_ku                                      (time_20_ku) float64 ...
    lat_20_c                                       (time_20_c) float64 ...
    lon_20_c                                       (time_20_c) float64 ...
Data variables: (12/276)
    UTC_day_01                                     (time_01) datetime64[ns] ...
    UTC_sec_01                                     (time_01) float64 ...
    UTC_day_20_ku                                  (time_20_ku) datetime64[ns] ...
    UTC_sec_20_ku                                  (time_20_ku) float64 ...
    UTC_day_20_c                                   (time_20_c) datetime64[ns] ...
    UTC_sec_20_c                                   (time_20_c) float64 ...
    ...                                             ...
    num_20hz_meas_01_ku                            (time_01) float32 ...
    num_20hz_meas_01_c                             (time_01) float32 ...
    orbit_type_01                                  (time_01) float32 ...
    waveform_qual_ice_20_ku                        (time_20_ku) float32 ...
    iono_cor_alt_filtered_01_ku                    (time_01) float32 ...
    iono_cor_alt_filtered_01_plrm_ku               (time_01) float32 ...
Attributes: (12/45)
    Conventions:                       CF-1.6
    title:                             IPF SRAL/MWR Level 2 Measurement
    mission_name:                      Sentinel 3B
    altimeter_sensor_name:             SRAL
    radiometer_sensor_name:            MWR
    gnss_sensor_name:                  GNSS
    ...                                ...
    xref_iono_data:                    S3B_SR_2_RGI_AX_20220218T000000_202202...
    xref_mog2d_data:                   S3__SR_2_PMO_AX_20220218T060000_202202...
    xref_seaice_concentration:         S3__SR_2_SIC_AX_20220209T000000_202202...
    xref_altimeter_ltm:                S3B_SR_1_CA1LAX_20000101T000000_202202...
    xref_doris_uso:                    S3B_SR_1_USO_AX_20180501T144459_202202...
    xref_time_correlation:             S3B_AX___FRO_AX_20220216T000000_202202...

<div class="alert alert-danger" role="alert">

## <a id='section3'></a>3. Applying your knowledge
[Back to top](#TOC_TOP)

</div>

<div class="alert alert-block alert-warning">

### Challenge:

What are differences you see between the level-1B and level-2 files? <div>

### Enter your solution here
* ...
* ...

* At level-1B all measurements are in one file (measurements.nc) and all acquisition parameters are in another file (acquisitions.nc). At level-2 all measurements are combined in three files; reduced_measurements.nc, standard_measurements.nc and enhanced_measurements.nc
* At level-1B the measurements consist of waveform echoes. At level-2 the measurements contain geophyiscal parameters.

<hr>
<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_1_SRAL_data_access_HDA_Data_Store.ipynb"><< Accessing SRAL data</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_3_SRAL_coverage.ipynb">Determining SRAL product coverage >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>